In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [119]:
# Set features. This will also be used as your x values.
# Started with 5 transit properties: period, time0bk, impact, duration, depth; scored .505/.487
# Added after first run: prad, teq, insol; scored .312/.316 - got worse
# Added TCE (threshold-crossing events) columns: model_snr, tce_plnt_num and Stellar params: steff, slogg, srad; scored .279/.290
# Removed 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol'
# Using highest correlations from the corr matrix:
# selected_features = df[['koi_prad', 'koi_impact', 'koi_time0bk', 'koi_model_snr', 'koi_depth', 'koi_fpflag_ss', 'koi_srad', 'koi_insol', 'koi_slogg', 'koi_kepmag']]

# All Features:
# selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]

# This feature set was .749/.757:
# selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_depth', 'koi_teq', 'koi_steff']]

# Top 10 features after RF analysis: .554/.569 with min/max, .757/759 with std scaler
selected_features = df[['koi_fpflag_nt', 'koi_model_snr', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_prad', 'koi_fpflag_ec', 'koi_depth', 'koi_period', 'koi_steff', 'koi_impact']]
selected_features.head()

,koi_fpflag_nt,koi_model_snr,koi_fpflag_co,koi_fpflag_ss,koi_prad,koi_fpflag_ec,koi_depth,koi_period,koi_steff,koi_impact
0,0,25.8,0,0,2.83,0,874.8,54.418383,5455,0.586
1,0,76.3,0,1,14.60,0,10829.0,19.899140,5853,0.969
2,0,505.6,0,1,33.46,0,8079.2,1.736952,5805,1.276
3,0,40.9,0,0,2.75,0,603.3,2.525592,6031,0.701
4,0,40.2,0,0,2.77,0,686.0,4.134435,6046,0.762


# Create a Train Test Split

Use `koi_disposition` for the y values

In [120]:
from sklearn.model_selection import train_test_split

X = selected_features
y = df['koi_disposition']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [121]:
X_train.head()

,koi_fpflag_nt,koi_model_snr,koi_fpflag_co,koi_fpflag_ss,koi_prad,koi_fpflag_ec,koi_depth,koi_period,koi_steff,koi_impact
6122,0,10.8,0,0,1.24,0,123.1,6.768901,5737,0.150
6370,0,13.8,0,1,0.86,1,114.6,0.733726,5855,0.291
2879,1,254.3,0,0,3.21,0,641.1,7.652707,6328,0.970
107,0,38.4,0,0,2.25,0,875.4,7.953547,4768,0.300
29,0,696.5,0,0,12.21,0,9802.0,4.959319,5712,0.831


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [122]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
# Scale your data
# X_scaler = MinMaxScaler().fit(X_train)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Train the Model



In [123]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model

LogisticRegression()

In [124]:
log_model.fit(X_train, y_train)

LogisticRegression()

In [125]:
print(f"Training Data Score: {log_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {log_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.5544535571237841
Testing Data Score: 0.5697940503432495


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [126]:
# Create the GridSearchCV model
from sklearn.svm import SVC
svc_model = SVC(kernel='linear')
svc_model

SVC(kernel='linear')

In [127]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[1,5,10,50], 'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svc_model, param_grid, verbose=3)

In [128]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.798, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.790, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.792, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.795, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.796, to

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [129]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.8153694176205619


# Save the Model

In [130]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model_saves/log_grid_exoplanet.sav'
joblib.dump(svc_model, filename)

['model_saves/log_grid_exoplanet.sav']